# Capstone Project: Predicting NHL Player Salary

## Part I - Project Intro and Data Collection

Author: Charles Ramey

Date: 05/15/2023

---

## Problem Statement

In the National Hockey League (NHL), team executives lack a robust, data-driven solution to estimate player salaries, which hinder's their ability to perform effective roster building and financial planning. This stems from the inherent complexity of factors that drive player salaries, including their performance, the quality of the team's they have played for, and the value of contracts signed by similar players. This project seeks to design a data-driven approach that can leverage historical data and advanced modeling techniques to help NHL executives balance their budgets, invest in their rosters, and remain competitive within the league.

#### Notebook Links

Part II - Exploratory Data Analysis (EDA)
- [`Part-2_eda.ipynb`](../code/Part-2_eda.ipynb)

Part III - Modeling
- [`Part-3.1_modeling-forwards.ipynb`](../code/Part-3.1_modeling-forwards.ipynb)
- [`Part-3.2_modeling-defense.ipynb`](../code/Part-3.2_modeling-defense.ipynb)
- [`Part-3.3_modeling-goalies.ipynb`](../code/Part-3.3_modeling-goalies.ipynb)

Part IV - Conclusion, Recommendations, and Sources
- [`Part-4_conclusion-and-recommendations.ipynb`](../code/Part-4_conclusion-and-recommendations.ipynb)

### Contents

- [Background](#Background)
- [Scraping & Cleaning: Signings](#Scraping-&-Cleaning:-Signings)
- [Scraping & Cleaning: Salary Cap](#Scraping-&-Cleaning:-Salary-Cap)
- [Scraping & Cleaning: Player Stats](#Scraping-&-Cleaning:-Player-Stats)
- [Scraping & Cleaning: Team Standings](#Scraping-&-Cleaning:-Team-Standings)
- [Merging Data](#Merging-Data)

## Background

In the NHL, player contracts have a fascinating rhythm. As each season passes, player agreements inevitably reach their expiration dates, transforming talented athletes into coveted free agents. And while many players would deny a desire for greater pay, they pour their hearts into improving their skills and contributing more to their teams, which helps to secure higher-paying contracts. As these players nagivate the uncertainty of free agency, NHL team executives are hard at work evaluating talent, both within and without the organization. Each year, teams are given a new [salary cap limit](https://www.sportingnews.com/us/nhl/news/nhl-salary-cap-rules-explained/x1wwiew656afzelhsx4tnecz) by the NHL, limiting the amount of money they can pay their players. Having a salary cap helps to keep the league fair and allows smaller market teams to remain competitive, however it challenges owners and general managers to piece together the most talented team possible on a limited budget.

In the 2021-22 season, the Colorado Avalanche achieved the ultimate goal: winning the Stanley Cup. A big reason for their success was a career-best performance by their second-line center, Nazem Kadri. The Avalanche traded for and signed Kadri in July of 2019, a couple years prior to their Stanley Cup run, inheriting Kadri's [\\$4.5M annual salary](https://www.capfriendly.com/players/nazem-kadri) from the Toronto Maple Leafs. In the two seasons that followed, Kadri did not eclipse 60 games played and never topped 36 points. However, all of that changed in Colorado's Stanley Cup season. Kadri amassed [87 points in 71 games](https://www.nhl.com/player/nazem-kadri-8475172), 26 more points than his previous career best. Kadri was instrumental in Colorado's championship push, and despite being a statistical outlier for Kadri's career, his season made him extremely valuable in a year when his contract was expiring and he was set to become a free agent. Colorado, having maxed out its available cap space, would have likely needed to move players in order to afford Kadri now. Despite the two parties wanting to stick together, Kadri ultimately landed a big payday through free agency, earning a [new contract](https://www.nhl.com/news/nazem-kadri-signs-seven-year-contract-with-flames/c-335317582) with the Calgary Flames that would pay him \\$7M annually, a more than 50% increase from his previous salary.

Nazem Kadri is not a unique case. The value of players is constantly changing based on their performance and other factors. As teams work to build a roster that can contend for the Stanley Cup, accurately valuing player contracts can help teams to find undervalued players, or potentially avoid overvalued players. This project seeks to create a [tool](https://nhl-salary-predictor.onrender.com/) that can take the guessing out of this process and help NHL executives plan ahead when players exceed, or fall short of, expectations.

### Disclaimer

During the creation of this notebook and the analysis contained herein, the author frequently referenced available documentation for imported libraries. Much of the coding syntax within this notebook is derived from various online sources, including stackoverflow, other public forums, and OpenAI's ChatGPT. The author does not claim the following code as fully original, and sources are cited where possible.

### Library Imports

In [1]:
import pandas as pd
import numpy as np
import time

from datetime import datetime

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import ElementClickInterceptedException
from selenium import __version__

import requests

All of the data collected for this project is scraped from various websites including [CapFriendly](https://www.capfriendly.com/), [MoneyPuck](https://moneypuck.com/), and [Hockey Reference](https://www.hockey-reference.com/). Selenium WebDriver with the Google Chrome WebDriver manager is used extensively in this first notebook to pull all the necessary data.

This notebook was originally run with Selenium v4.8.2. Please use the code block below to check the version of Selenium that you are currently running this notebook with.

In [2]:
print(f"Selenium version: {__version__}")

Selenium version: 4.8.2


Before we can start scraping data from the web, we need to initiate the Selenium webdriver. We'll save it as `driver` so we can call it throughout the notebook.

In [3]:
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

One other thing we'll do now before beginning is define a dictionary that contains all NHL teams dating back to the 2010-11 season, paired with their associated three-letter team name abbreviation which will be used to standardize team naming across datasets pulled from different sources.

Note that not all of these teams were active through the period of time that this analysis considers. In particular, the Atlanta Thrashers (ATL) played their final season in 2010-11 before relocating and becoming the Winnipeg Jets (WPG) the following season. More recently, the NHL has expanded twice, adding the Vegas Golden Knights (VGK) in the 2017-18 season and the Seattle Kraken (SEA) in the 2021-22 season.

In [4]:
teams_dict = {
    'Anaheim Ducks'        : 'ANA',
    'Arizona Coyotes'      : 'ARI',
    'Atlanta Thrashers'    : 'ATL',    # Winnipeg Jets after 2010-11 season
    'Boston Bruins'        : 'BOS',
    'Buffalo Sabres'       : 'BUF',
    'Calgary Flames'       : 'CGY',
    'Carolina Hurricanes'  : 'CAR',
    'Chicago Blackhawks'   : 'CHI',
    'Colorado Avalanche'   : 'COL',
    'Columbus Blue Jackets': 'CBJ',
    'Dallas Stars'         : 'DAL',
    'Detroit Red Wings'    : 'DET',
    'Edmonton Oilers'      : 'EDM',
    'Florida Panthers'     : 'FLA',
    'Los Angeles Kings'    : 'LAK',
    'Minnesota Wild'       : 'MIN',
    'Montreal Canadiens'   : 'MTL',
    'Nashville Predators'  : 'NSH',
    'New Jersey Devils'    : 'NJD',
    'New York Islanders'   : 'NYI',
    'New York Rangers'     : 'NYR',
    'Ottawa Senators'      : 'OTT',
    'Philadelphia Flyers'  : 'PHI',
    'Phoenix Coyotes'      : 'ARI',
    'Pittsburgh Penguins'  : 'PIT',
    'San Jose Sharks'      : 'SJS',
    'Seattle Kraken'       : 'SEA',    # Added in 2021-22 season
    'St. Louis Blues'      : 'STL',
    'Tampa Bay Lightning'  : 'TBL',
    'Toronto Maple Leafs'  : 'TOR',
    'Vancouver Canucks'    : 'VAN',
    'Vegas Golden Knights' : 'VGK',    # Added in 2017-18 season
    'Washington Capitals'  : 'WSH',
    'Winnipeg Jets'        : 'WPG'
}

---
## Scraping & Cleaning: Signings

In this section, we're going to scrape CapFriendly for player signings dating back to the conclusion of the 2010-11 season. The reason we're only getting signings back to 2011 instead of 2010 is because we are makig the assumption that the player's stats from the 2010-11 season drive the value of the contract they sign for the 2011-12 season. This assumption is fundamental to the simplication of the modeling process, which I will dicuss further in the modeling notebook (Part III). For now, note that all player signings between the conclusion of one season and the conclusion of the following season are assumed to be driven by the player's stats from the last complete season.

I have included the gifs below to help visualize the order of operations for the scraping of player signings, since this part of the code is the most complex. When first interacting with the web page, up to 300 results are displayed in the table as long as a user repeatedly scrolls to the bottom of the page to load more results. However, after some trial and error, I discovered that the page starts limiting the table to 50 results, without the options to load more by scrolling or moving to a next page. As a result, the only way to get all signings, the search needed to be iteratively refined and narrowed to display only signings for one position at a time, month by month, keeping the results below 50 per month.

In the first `for` loop, we are selecting the player position for which we want to scrape signings. For the Forwards position, the driver will get the signings for January of the first year (2011), then loop through the remaining months in 2011.

<img src="../assets/first_selection_looped.gif" alt="first_selection_looped" style="width:800px;height:250px;">

When the code has finished looping through the months of the first year, it will then select the next year. However, as you can see in the gif below, in order to move forward to the next year the driver needs to start with the `to_year` since the `from_year` does not present the option to move *past* the `to_year`, which is still currently 2011. As a result, we he progress the `to_year` to January 31st of the next year, followed by the `from_year` to January 1st of the next year. We then repeat looping through the months of the next year. At this point, we are still collecting data for Forwards and have not begun getting data fro Defense or Goalies.

<img src="../assets/next_year_looped.gif" alt="next_year_looped" style="width:800px;height:250px;">

When the code has finished looping through all months for all years from 2011 to 2022, we move to the next position, Defense. Similar to how we could not progress the `from_year` without first progressing the `to_year`, we cannot reset the `to_year` to 2011 without first resetting the `from_year`. This is ultimately why the code must loop through the months of the first year separately. While the `to_year` needs to be selected first for the remainder of the years, the `from_year` needs to be selected first when starting again with a new position. 

<img src="../assets/next_position_looped.gif" alt="next_position_looped" style="width:800px;height:250px;">

The dictionary below is necessary in order for the code that scrapes CapFriendly to work properly. We will be primarily selecting different web elements by their XPath. Thus, when looping through the selection of different options for the same element, we need to identify the specific **option** number as it would appear for that element given the state of the web page.

For example, `from_years` defines all of the dropdown options that will be selected throughout the scraping process, and since there is nothing limiting how early we set the `from_year`, each option number is relative to the earliest possible year. In this case, the year 2000 is `option[1]` and the year 2011 is `option[12]`. Therefor, the `from_years` option numbers range from 13 to 23 (2011 is scraped separately).

`//*[@id="ui-datepicker-div"]/div/div/select[2]/option[12]`

However, since the dropdown to select the end of the date range (`to_year`) only displays the years available after the `from_year`, the option to select the next year is always `option [2]`, as the current year in the loop will always be `option [1]`. The same logic must also be applied to the months, thus why the `from_months` count up from `option [2]` while the `to_months` are all `option [2]`.

In [5]:
positions = {
    'forwards': '2',
    'defense': '6',
    'goaltender': '7'
}

to_years = {
    '2012': '2',
    '2013': '2',
    '2014': '2',
    '2015': '2',
    '2016': '2',
    '2017': '2',
    '2018': '2',
    '2019': '2',
    '2020': '2',
    '2021': '2',
    '2022': '2'
}

from_years = {
    '2012': '13',
    '2013': '14',
    '2014': '15',
    '2015': '16',
    '2016': '17',
    '2017': '18',
    '2018': '19',
    '2019': '20',
    '2020': '21',
    '2021': '22',
    '2022': '23'
}

to_months = {
    'Feb': '2',
    'Mar': '2',
    'Apr': '2',
    'May': '2',
    'Jun': '2',
    'Jul': '2',
    'Aug': '2',
    'Sep': '2',
    'Oct': '2',
    'Nov': '2',
    'Dec': '2'
}

from_months = {
    'Feb': '2',
    'Mar': '3',
    'Apr': '4',
    'May': '5',
    'Jun': '6',
    'Jul': '7',
    'Aug': '8',
    'Sep': '9',
    'Oct': '10',
    'Nov': '11',
    'Dec': '12'
}

To reduce the chances of the driver attempting to click an element before it has loaded, we need to add a brief delay with `time.sleep( )` before each click. This can cause the code to get even bulkier than it already is, therefor the function below is defined to try to reduce the overall lines of code by building the pause and the click into a sigle line. This function is used throughout the notebook for the webscraping process.

In [6]:
def select_element(xpath):
    time.sleep(0.1)
    driver.find_element(By.XPATH, xpath).click()

In [7]:
driver.get("https://www.capfriendly.com/signings")
time.sleep(2)

# Define the columns for the final signings spreadsheet
# This will prevent appending unwanted or duplicate columns
columns = ['PLAYER','AGE','POS','TEAM','DATE','TYPE','EXTENSION',
           'STRUCTURE','LENGTH','VALUE','CAP HIT']

# Initiate final dataframe to store all signings
signings = pd.DataFrame(columns=columns)

for pos in positions.values():
    # Select Position Box (Forwards/Defense)
    select_element(f'//*[@id="pos"]/option[{pos}]')
    # Select Date Range Box (FROM)
    select_element('//*[@id="from"]')
    # Select First Year, 2010 (FROM)
    select_element('//*[@id="ui-datepicker-div"]/div/div/select[2]/option[12]')
    # Select First Month, Jan (FROM)
    select_element('//*[@id="ui-datepicker-div"]/div/div/select[1]/option[1]')
    # Select Day (first of the month)
    time.sleep(0.1)
    first_row = driver.find_elements(By.XPATH, '//*[@id="ui-datepicker-div"]/table/tbody/tr[1]/td')
    # Loop through row of date elements until one is clickable
    for td_element in first_row:
        try:
            td_element.click()
            break
        except ElementClickInterceptedException:
            continue    
    
    # Select Date Range Box (TO)
    select_element('//*[@id="to"]')
    # Select First Year, 2010 (TO)
    select_element('//*[@id="ui-datepicker-div"]/div/div/select[2]/option[1]')
    # Select First Month, Jan (TO)
    select_element('//*[@id="ui-datepicker-div"]/div/div/select[1]/option[1]')
    # Select Day (last of the month)
    time.sleep(0.1)
    last_row = driver.find_elements(By.XPATH, '//*[@id="ui-datepicker-div"]/table/tbody/tr[last()]/td')
    # Loop through row of date elements until one is clickable
    for td_element in last_row[::-1]:
        try:
            td_element.click()
            break
        except ElementClickInterceptedException:
            continue
    
    # Save Result of First Month
    time.sleep(0.1)
    table_element = driver.find_element(By.XPATH, '//*[@id="na"]')
    table_data = pd.read_html(table_element.get_attribute('outerHTML'))
    jan_2011 = pd.DataFrame(table_data[0])
    signings = pd.concat([signings, jan_2011], join='inner')    
    
    
    # Iterate through all remaining months for the first year
    for (to_month_str, to_month), (from_month_str, from_month) in zip (to_months.items(), from_months.items()):
        # Select Date Range Box (TO)
        select_element('//*[@id="to"]')
        # Select Month (TO)
        select_element(f'//*[@id="ui-datepicker-div"]/div/div/select[1]/option[{to_month}]')
        # Select Day (last of the month)
        time.sleep(0.1)
        last_row = driver.find_elements(By.XPATH, '//*[@id="ui-datepicker-div"]/table/tbody/tr[last()]/td')
        # Loop through row of date elements until one is clickable
        for td_element in last_row[::-1]:
            try:
                td_element.click()
                break
            except ElementClickInterceptedException:
                continue

        # Select Date Range Box (FROM)
        select_element('//*[@id="from"]')        
        # Select Month (FROM)
        select_element(f'//*[@id="ui-datepicker-div"]/div/div/select[1]/option[{from_month}]')
        # Select Day (first of the month)
        time.sleep(0.1)
        first_row = driver.find_elements(By.XPATH, '//*[@id="ui-datepicker-div"]/table/tbody/tr[1]/td')
        # Loop through row of date elements until one is clickable
        for td_element in first_row:
            try:
                td_element.click()
                break
            except ElementClickInterceptedException:
                continue        
    
        # Save Result of Each Month for First Year
        time.sleep(0.1)
        table_element = driver.find_element(By.XPATH, '//*[@id="na"]')
        table_data = pd.read_html(table_element.get_attribute('outerHTML'))
        month_2011 = pd.DataFrame(table_data[0])
        signings = pd.concat([signings, month_2011], join='inner')
    
    
    # Iterate through remaining years to get all signings for position
    for (to_year_str, to_year), (from_year_str, from_year) in zip (to_years.items(), from_years.items()):        
        # Select Date Range Box (TO)
        select_element('//*[@id="to"]')
        # Select Year (TO)
        select_element(f'//*[@id="ui-datepicker-div"]/div/div/select[2]/option[{to_year}]')
        # Select Month (TO)
        select_element('//*[@id="ui-datepicker-div"]/div/div/select[1]/option[1]')
        # Select Day (last of the month)
        time.sleep(0.1)
        last_row = driver.find_elements(By.XPATH, '//*[@id="ui-datepicker-div"]/table/tbody/tr[last()]/td')
        # Loop through row of date elements until one is clickable
        for td_element in last_row[::-1]:
            try:
                td_element.click()
                break
            except ElementClickInterceptedException:
                continue
                  
        # Select Date Range Box (FROM)
        select_element('//*[@id="from"]')
        # Select Year (FROM)
        select_element(f'//*[@id="ui-datepicker-div"]/div/div/select[2]/option[{from_year}]')
        # Select Month (FROM)
        select_element('//*[@id="ui-datepicker-div"]/div/div/select[1]/option[1]')
        # Select Day (first of the month)
        time.sleep(0.1)
        first_row = driver.find_elements(By.XPATH, '//*[@id="ui-datepicker-div"]/table/tbody/tr[1]/td')
        # Loop through row of date elements until one is clickable
        for td_element in first_row:
            try:
                td_element.click()
                break
            except ElementClickInterceptedException:
                continue
        
        # Save result of First Month for Each Year
        time.sleep(0.1)
        table_element = driver.find_element(By.XPATH, '//*[@id="na"]')
        table_data = pd.read_html(table_element.get_attribute('outerHTML'))
        jan_year = pd.DataFrame(table_data[0])
        signings = pd.concat([signings, jan_year], join='inner')
        

        for (to_month_str, to_month), (from_month_str, from_month) in zip (to_months.items(), from_months.items()):
            # Select Date Range Box (TO)
            select_element('//*[@id="to"]')
            # Select Month (TO)
            select_element(f'//*[@id="ui-datepicker-div"]/div/div/select[1]/option[{to_month}]')
            # Select Day (last of the month)
            time.sleep(0.1)
            last_row = driver.find_elements(By.XPATH, '//*[@id="ui-datepicker-div"]/table/tbody/tr[last()]/td')
            # Loop through row of date elements until one is clickable
            for td_element in last_row[::-1]:
                try:
                    td_element.click()
                    break
                except ElementClickInterceptedException:
                    continue

            # Select Date Range Box (FROM)
            select_element('//*[@id="from"]')        
            # Select Month (FROM)
            select_element(f'//*[@id="ui-datepicker-div"]/div/div/select[1]/option[{from_month}]')
            # Select Day (first of the month)
            time.sleep(0.1)
            first_row = driver.find_elements(By.XPATH, '//*[@id="ui-datepicker-div"]/table/tbody/tr[1]/td')
            # Loop through row of date elements until one is clickable
            for td_element in first_row:
                try:
                    td_element.click()
                    break
                except ElementClickInterceptedException:
                    continue

            # Save result of Each Month for Each Remaining Year
            time.sleep(0.1)
            table_element = driver.find_element(By.XPATH, '//*[@id="na"]')
            table_data = pd.read_html(table_element.get_attribute('outerHTML'))
            month_year = pd.DataFrame(table_data[0])
            signings = pd.concat([signings, month_year], join='inner')

# View the size of the final dataframe
print(f'Shape of signings: {signings.shape}')

Shape of signings: (3734, 11)


In [8]:
signings.head()

,PLAYER,AGE,POS,TEAM,DATE,TYPE,EXTENSION,STRUCTURE,LENGTH,VALUE,CAP HIT
0,Nate Thompson,26,C,TBL,"Jan. 31, 2011",Standard,✔,1-way,2,"$1,800,000","$900,000"
1,Matt Moulson,27,LW,NYI,"Jan. 27, 2011",Standard,✔,1-way,3,"$9,400,000","$3,133,333"
2,Alexander Semin,26,"RW, LW",WSH,"Jan. 27, 2011",Standard,✔,1-way,1,"$6,700,000","$6,700,000"
3,Mark Letestu,25,"C, RW",PIT,"Jan. 18, 2011",Standard,✔,1-way,2,"$1,250,000","$625,000"
4,Kyle Wellwood,27,C,ARI,"Jan. 17, 2011",Standard,NaN,2-way,1,"$650,000","$650,000"


Right away, theres a few things we want to do to clean up our dataframe and data. First, we want to snake-case all of the headers to make them easier to reference and work with. Positions are also broken up (i.e. Forwards are divided into C, LW, and RW while Defense is divided into LD and RD), so we want to group them into just Forwards (F), Defense (D), and Goalies (G). Right now we also have both 1-way and 2-way contracts. For context, 2-way contracts are when a player is signed to play for both an NHL team and its AHL farm team. As such, 2-way contracts are almost exclusively the NHL minimum salary. Keeping these contracts would significantly skew the average contract value towards the minimum, so we're soing to remove them as well.

In [9]:
# Snake-case headers
signings.columns = signings.columns.str.lower().str.replace(' ', '_')

# Reduce position lables to F, D, and G
signings['pos'] = np.where(signings['pos'].str.contains('D'), 'D',
                           np.where(signings['pos'].str.contains('G'), 'G', 'F'))

# Remove 2-way contracts
signings = signings[signings['structure'] == '1-way']

While monetary symbols are informative, the contract value is currently represented by a string-type value, and we will need to convert it to a numeric value to be used in modeling. The value is also currently represented as the total contract value. To standardize contracts, we need to divide the total value by the length in yeats to the the contract average annual values (`contract_aav`). This will also replace the cap hit of a contract which is similar to `contract_aav`, except that it may vary from year to year if a team decides they want different portions of a players contract to count against the team salary cap in different seasons.

In [10]:
# Convert the value column to a numeric datatype
for col in ['value']:
    signings.loc[:, col] = signings[col].replace('[\$,]', '', regex=True).astype(int)

In [11]:
# Create the contract_aav column by dividing contract value by contract lenght
signings.loc[:, 'contract_aav'] = signings['value'] / signings['length']
signings.loc[:, 'contract_aav'] = signings['contract_aav'].astype(int)
signings = signings.drop(columns=['age', 'team', 'extension', 'type', 'length', 'value', 'cap_hit', 'structure'])

Right now, each signing is associated with the specific date that the contract was signed, however, in order to tie a player's contract to their performance, we're going to break them up by season. As mentioned before, to simplify this analysis, we're going to assume that the contract value is driven by the previous season's performance. Thus, all players signed between the conclusion of the 2010-11 season and the conclusion of the 2011-12 season will be labeled as `2010-11`, since their contracts will be linked to their performance in the 2010-11 season.

To do this, we first need to convert the date column into a datetime value that we can use. We will then filter out any 2011 contracts that were signed before the end of the 2010-11 season. And lastly, we'll make the conversion from date to season.

In [12]:
# Define a function that will convert the date string to a datetime value
def convert_to_datetime(date_string):
    try:
        # Try converting with the first format 'Mar. 3, 2011'
        date_obj = datetime.strptime(date_string, '%b. %d, %Y')
    except ValueError:
        # If the first format fails, try converting with the second format 'Mar 3, 2011'
        date_obj = datetime.strptime(date_string, '%b %d, %Y')
    return date_obj

# Apply the conversion function to the entire column
signings['date'] = signings['date'].apply(convert_to_datetime)

signings = signings[signings['date'] > '2011-06-15']

In [13]:
# Check the new format
signings.head()

,player,pos,date,contract_aav
1,Dana Tyrell,F,2011-11-28,600000
2,Kyle Turris,F,2011-11-22,1400000
18,Shawn Thornton,F,2012-03-17,1100000
23,Daniel Carcillo,F,2012-03-12,825000
26,Mikhail Grabovski,F,2012-03-06,5500000


In [14]:
# Reset the index of the dataframe
signings = signings.reset_index(drop=True)

In [15]:
# https://en.wikipedia.org/wiki/List_of_NHL_seasons
season_ranges = [
    {'season': '2010-11', 'start': pd.to_datetime('2011-06-15'), 'end': pd.to_datetime('2012-06-11')},
    {'season': '2011-12', 'start': pd.to_datetime('2012-06-11'), 'end': pd.to_datetime('2013-06-24')},
    {'season': '2012-13', 'start': pd.to_datetime('2013-06-24'), 'end': pd.to_datetime('2014-06-13')},
    {'season': '2013-14', 'start': pd.to_datetime('2014-06-13'), 'end': pd.to_datetime('2015-06-15')},
    {'season': '2014-15', 'start': pd.to_datetime('2015-06-15'), 'end': pd.to_datetime('2016-06-12')},
    {'season': '2015-16', 'start': pd.to_datetime('2016-06-12'), 'end': pd.to_datetime('2017-06-11')},
    {'season': '2016-17', 'start': pd.to_datetime('2017-06-11'), 'end': pd.to_datetime('2018-06-07')},
    {'season': '2017-18', 'start': pd.to_datetime('2018-06-07'), 'end': pd.to_datetime('2019-06-12')},
    {'season': '2018-19', 'start': pd.to_datetime('2019-06-12'), 'end': pd.to_datetime('2020-09-28')},
    {'season': '2019-20', 'start': pd.to_datetime('2020-09-28'), 'end': pd.to_datetime('2021-07-07')},
    {'season': '2020-21', 'start': pd.to_datetime('2021-07-07'), 'end': pd.to_datetime('2022-06-26')},
    {'season': '2021-22', 'start': pd.to_datetime('2022-06-26'), 'end': pd.to_datetime('2022-12-31')}
]

# Create the empyty 'season' column
signings['season'] = ''

# Iterate through the date column and assign the corresponding label for the date range to the new 'season' column
for i in range(len(signings)):
    for season_range in season_ranges:
        if season_range['start'] < signings.loc[i, 'date'] <= season_range['end']:
            signings.loc[i, 'season'] = season_range['season']
            break   # Exit the inner loop once a matching date range is found

signings = signings.drop(columns='date')

In [16]:
# Check the final dataframe
signings

,player,pos,contract_aav,season
0,Dana Tyrell,F,600000,2010-11
1,Kyle Turris,F,1400000,2010-11
2,Shawn Thornton,F,1100000,2010-11
3,Daniel Carcillo,F,825000,2010-11
4,Mikhail Grabovski,F,5500000,2010-11
...,...,...,...,...
1319,Spencer Martin,G,762500,2020-21
1320,Joonas Korpisalo,G,1300000,2020-21
1321,Daniel Vladar,G,2200000,2021-22
1322,Jake Allen,G,3850000,2021-22


In [17]:
# Save cleaned signings dataframe
signings.to_csv('../data/signings_cleaned.csv', index=False)

To make it easier to merge skater signings with skater stats and goalie signings with goalie stats, we're going to create two new dataframes called `signings_skaters` and `signings_goalies` to use later on.

In [18]:
signings_skaters = signings[signings['pos'] != 'G']

signings_goalies = signings[signings['pos'] == 'G']

---
## Scraping & Cleaning: Salary Cap

Next, we're going to get the NHL's salary cap limits for each year, including the upper limit, lower limit, and the minimum salary that a player can be paid. These values are expected to have a significant impact on our model's predictions of player salaries. In particular, with a significant number of players signing minimum value contracts, the minimum salary is likely directly correlated to contract value.

The previous webscraping task was fairly time-intensive, so before we continue, we'll quit out of the webdriver and re-initiate it. This will help us to avoid timing out and interrupting the webscraping process.

In [19]:
driver.quit()
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

Fortunately, all of the salary cap data is contained within a single table on a single page, so we're able to collect all of the data with only a few lines of code below.

In [20]:
driver.get("https://www.capfriendly.com/salary-cap")
time.sleep(2)

# Get table information and save to a dataframe
table_element = driver.find_element(By.XPATH, '//*[@id="salaryCapHistoryInnerContainer"]/table')
table_data = pd.read_html(table_element.get_attribute('outerHTML'))
salary_cap = pd.DataFrame(table_data[0])

In [21]:
# Take a look at the data
salary_cap

,SEASON,CONFIRMED,% CHANGE,UPPER LIMIT,LOWER LIMIT,MIN. SALARY
0,2025-26,NHL Estimate,5.14%,"$92,000,000","$68,000,000","$775,000"
1,2024-25,NHL Estimate,4.79%,"$87,500,000","$64,700,000","$775,000"
2,2023-24,NHL Estimate,1.21%,"$83,500,000","$61,700,000","$775,000"
3,2022-23,"Mar. 29, 2022",1.23%,"$82,500,000","$61,000,000","$750,000"
4,2021-22,"Jul. 1, 2021",0.00%,"$81,500,000","$60,200,000","$750,000"
5,2020-21,"Jul. 10, 2020",0.00%,"$81,500,000","$60,200,000","$700,000"
6,2019-20,"Jun. 22, 2019",2.52%,"$81,500,000","$60,200,000","$700,000"
7,2018-19,"Jun. 21, 2018",6.00%,"$79,500,000","$58,800,000","$650,000"
8,2017-18,"Jun. 18, 2017",2.74%,"$75,000,000","$55,400,000","$650,000"
9,2016-17,"Jun. 21, 2016",2.24%,"$73,000,000","$54,000,000","$575,000"


Once again, we're going to want to snake-case the headers for usabilility and consistency. We will also need to convert some string values to numeric again. We will convert the dollar value columns to numeric as we did before, but this time we also need to convert the percent values to numeric values as well since they are currently represented as strings.

In [22]:
# Convert headers to snake-case
salary_cap.columns = salary_cap.columns.str.lower().str.replace(' ', '_', regex=False).str.replace('%','pct', regex=False).str.replace('.', '', regex=False)

In [23]:
# Convert percent values (string) to numeric
salary_cap['pct_change'] = salary_cap['pct_change'].str.strip('%').astype(float) / 100

In [24]:
# Convert monetary columns (string) to numeric
for col in ['upper_limit', 'lower_limit', 'min_salary']:
    salary_cap[col] = salary_cap[col].replace('[\$,]', '', regex=True).astype(int)

We have a few more seasons than we are considering in this analysis, we we're going to go ahead and drop the season that will not be used. We also don't need to know the date that the NHL confirmed the salary cap limits, since we're grouping data by season, so we will drop the `confirmed` column as well.

In [25]:
# Define seasons to be dropped from dataframe
seasons_to_exclude = [
    '2005-06',
    '2006-07',
    '2007-08',
    '2008-09',
    '2009-10',
    '2010-11',
    '2024-25',
    '2025-26'
]

# Drop unused seasons
salary_cap = salary_cap[~salary_cap['season'].isin(seasons_to_exclude)]

# Drop confrimed column
salary_cap = salary_cap.drop(columns='confirmed')

Note that the `season` column represents the upcoming season for which the salary cap limit is being set. Therefor, if we want these values to be used to predict salary alongside play stats from the previous season, we need to switch each season to the prvious season. For example, while the salary cap is set for the 2011-12 season, we need to convert it to 2010-11 so it can be merged with a player's stats from the 2010-11 season.

In [26]:
# Define season pairs to convert season to previous season
cap_to_stats = {
    '2011-12': '2010-11',
    '2012-13': '2011-12',
    '2013-14': '2012-13',
    '2014-15': '2013-14',
    '2015-16': '2014-15',
    '2016-17': '2015-16',
    '2017-18': '2016-17',
    '2018-19': '2017-18',
    '2019-20': '2018-19',
    '2020-21': '2019-20',
    '2021-22': '2020-21',
    '2022-23': '2021-22',
    '2023-24': '2022-23'
}

# Convert season to previous season
salary_cap['season'] = salary_cap['season'].map(cap_to_stats)

In [27]:
# Check final dataframe
salary_cap

,season,pct_change,upper_limit,lower_limit,min_salary
2,2022-23,0.0121,83500000,61700000,775000
3,2021-22,0.0123,82500000,61000000,750000
4,2020-21,0.0000,81500000,60200000,750000
5,2019-20,0.0000,81500000,60200000,700000
6,2018-19,0.0252,81500000,60200000,700000
7,2017-18,0.0600,79500000,58800000,650000
8,2016-17,0.0274,75000000,55400000,650000
9,2015-16,0.0224,73000000,54000000,575000
10,2014-15,0.0348,71400000,52800000,575000
11,2013-14,0.0731,69000000,51000000,525000


In [28]:
# Save cleaned salary_cap dataframe
salary_cap.to_csv('../data/salary_cap_cleaned.csv', index=False)

---
## Scraping & Cleaning: Player Stats

In this section, we will scrape stats for players by season from MoneyPuck.com. Since skaters (forwards & defense) are measured by different statistical catergories than goalies, this section will have two parts, one for skaters and one for goalies.

Like the `signings` data, scraping player stats will require looping through dropdown options for seasons. Each season will have have a different option and the code will loop through the option values to get the correct XPath for each season.

<img src="../assets/moneypuck-dropdown.png" alt="moneypuck-dropdown" style="width:600px;height:350px;">


In [29]:
# Define season / element option pairs
seasons = {
    '2010-11': '13',
    '2011-12': '12',
    '2012-13': '11',
    '2013-14': '10',
    '2014-15': '9',
    '2015-16': '8',
    '2016-17': '7',
    '2017-18': '6',
    '2018-19': '5',
    '2019-20': '4',
    '2020-21': '3',
    '2021-22': '2',
    '2022-23': '1'
}

### Skater States (Forwards & Defense)

To be cautious and avoid timing out, we'll quit and re-initiate the driver again before we start scraping again.

In [30]:
driver.quit()
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

There are a couple things we're going to do differently with this code. First, since MoneyPuck does not have a specific column for the team name but rather has an image of the team logo for each player, we need to dig into the attributes of the logos to get the team names and append these to the stats dataframe. Second, we're going to build in the snake-casing step. And last, MoneyPuck also does not have a season or date column in the table, so we need to append the season to the dataframe from the dictionary we defined above.

In [31]:
driver.get("https://moneypuck.com/stats.htm")
time.sleep(3)

# Initiate final dataframe to store all player stats
skater_stats = pd.DataFrame()

# Select only Regular Season Stats
select_element('//*[@id="table_playoff_type"]/option[1]')

# Iterate through all seasons from 2010-11 to 2021-22
for season_str, season in seasons.items():
    # Select Season
    select_element(f'//*[@id="season_type"]/option[{season}]')
    
    # Save the stas from the current season
    time.sleep(8)
    table_element = driver.find_element(By.XPATH, '//*[@id="includedContent"]/table')
    table_data = pd.read_html(table_element.get_attribute('outerHTML'))
    season_stats = pd.DataFrame(table_data[0])
    
    # Get team names from logo attributes
    tr_elements = driver.find_elements(By.XPATH, '//*[@id="includedContent"]/table/tbody/tr')
    alt_values = [tr_element.find_element(By.XPATH, './th/table/tbody/tr/td[2]/img').get_attribute('alt') for tr_element in tr_elements]
    
    # Convert headers to snake-case
    season_stats.columns = season_stats.columns.str.lower().str.replace(' ', '_').str.replace('%','pct')
    
    # The produced dataframe added a null row every other row, we'll drop those using the position column
    season_stats.dropna(subset=['pos'], inplace=True)

    # Add season and team columns
    season_stats['season'] = season_str
    season_stats['team'] = alt_values
    season_stats['team'] = season_stats['team'].map(teams_dict)
    
    # Move new columns to the beginning of the dataframe for clarity
    season = season_stats.pop('season')
    team = season_stats.pop('team')
    season_stats.insert(2, 'season', season)
    season_stats.insert(3, 'team', team)
    
    # Print shape of each season to view scraping progress
    print(f'Shape of {season_str} stats: {season_stats.shape}')
    
    # Append each season to the main stats dataframe
    skater_stats = pd.concat([skater_stats, season_stats])

Shape of 2010-11 stats: (887, 80)
Shape of 2011-12 stats: (890, 80)
Shape of 2012-13 stats: (837, 80)
Shape of 2013-14 stats: (882, 80)
Shape of 2014-15 stats: (879, 80)
Shape of 2015-16 stats: (897, 80)
Shape of 2016-17 stats: (884, 80)
Shape of 2017-18 stats: (886, 80)
Shape of 2018-19 stats: (903, 80)
Shape of 2019-20 stats: (880, 80)
Shape of 2020-21 stats: (910, 80)
Shape of 2021-22 stats: (998, 80)
Shape of 2022-23 stats: (949, 80)


In [32]:
skater_stats

,name,pos,season,team,games_played,icetime_(minutes),expected_goals,goals,assists,points,...,share_of_xgoals_from_rebounds_shots,xgoals_from_non_rebounds,xgoals_of_expected_rebounds,created_xgoals,created_xgoals_minus_actual_xgoals,goals.1,expected_goals.1,shooting_talent_above_average,shooting_talent_adjusted_expected_goals,goals_above_shooting_talent
0,1Frans Nielsen,C,2010-11,NYI,71,1261.0,14.7,13.0,31.0,44.0,...,16.2%,12.4,2.1,14.4,-0.3,13.0,14.7,-17%,12.2,0.8
2,2Jaroslav Spacek,D,2010-11,MTL,59,1135.0,2.7,1.0,15.0,16.0,...,2.2%,2.6,0.9,3.5,0.8,1.0,2.7,0%,2.7,-1.7
4,3Antti Miettinen,R,2010-11,MIN,73,1242.0,15.7,16.0,19.0,35.0,...,7.6%,14.5,2.6,17.1,1.4,16.0,15.7,0.6%,15.8,0.2
6,4Kyle Quincey,D,2010-11,COL,21,411.0,1.4,0.0,1.0,1.0,...,3.6%,1.3,0.5,1.8,0.4,0.0,1.4,-22.6%,1.1,-1.1
8,5Sergei Samsonov,L,2010-11,FLA,78,1207.0,15.6,13.0,27.0,40.0,...,28.2%,11.2,2.1,13.3,-2.3,13.0,15.6,4.4%,16.2,-3.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1888,945Rasmus Asplund,C,2022-23,NSH,46,504.0,5.9,2.0,6.0,8.0,...,41.4%,3.4,0.7,4.2,-1.7,2.0,5.9,-21.6%,4.6,-2.6
1890,946Ross Johnston,L,2022-23,NYI,16,124.0,0.3,0.0,2.0,2.0,...,0%,0.3,0.1,0.4,0.1,0.0,0.3,0%,0.3,-0.3
1892,947Neal Pionk,D,2022-23,WPG,82,1799.0,7.0,10.0,23.0,33.0,...,9.7%,6.3,1.8,8.1,1.1,10.0,7.0,-12.5%,6.1,3.9
1894,948Tyler Pitlick,C,2022-23,STL,61,614.0,5.6,7.0,9.0,16.0,...,30.3%,3.9,0.8,4.7,-0.9,7.0,5.6,3.8%,5.9,1.1


We have some columns for stats that are measured as percentages. Most of these columns headers have the '%' symbol in the header, and the values are represented as strings. We'll convert the '%' symbol to the alphanumeric 'pct' and convert the percentage strings to numeric values.

In [33]:
# Change headers to alphanumeric and convert percentage strings to numeric values
pct_cols = [col for col in skater_stats.columns if 'pct' in col]
for col in pct_cols:
    skater_stats[col] = skater_stats[col].str.strip('%').astype(float) / 100

In [34]:
# Look to see what other object columns are in the dataframe
skater_stats.select_dtypes(include=['object']).columns.tolist()

['name',
 'pos',
 'season',
 'team',
 'games_played',
 'share_of_possible_icetime',
 'share_of_xgoals_from_rebounds_shots',
 'shooting_talent_above_average']

Games played should be an integrer value, and we have a couple percentage-based columns that snuck through, so we will go ahead and convert these to the appropriate datatypes manually.

In [35]:
skater_stats['games_played'] = skater_stats['games_played'].astype(int)
skater_stats['share_of_possible_icetime'] = skater_stats['share_of_possible_icetime'].str.strip('%').astype(float) / 100
skater_stats['share_of_xgoals_from_rebounds_shots'] = skater_stats['share_of_xgoals_from_rebounds_shots'].str.strip('%').astype(float) / 100
skater_stats['shooting_talent_above_average'] = skater_stats['shooting_talent_above_average'].str.strip('%').astype(float) / 100

Looking through the dataframe, we can see that many columns are represented as float values despite only containing round numbers. While this shouldn't cause any issues, it is good practice to change them to integer datatypes, which use less memory than floats. 

In [36]:
# Identify columns that contain round float values
int_cols = ['games_played','icetime_(minutes)','goals','assists',
           'points','primary_assists','secondary_assists','shifts',
           'hits','pim','pim_drawn','pim_differential','shots_blocked_by_player',
           'takeaways','giveaways', 'defensive_zone_giveaways','shot_attempts',
           'shots_on_goal','shots_that_missed_net','shots_that_were_blocked',
           'high_danger_unblocked_shot_attempts','medium_danger_unblocked_shot_attempts',
           'low_danger_unblocked_shot_attempts','on-ice_goal_differential',
           'rebounds_created']

# Convert round float values to integers
for col in int_cols:
    skater_stats[col] = skater_stats[col].astype(int)

While looking through the columns, we also find that the `goals` and `expected_goals` columns were duplicated. Let's drop those duplicates.

In [37]:
skater_stats = skater_stats.drop(columns=['goals.1', 'expected_goals.1'])

For some reason, the player names appear in a bad format with a number preceding the name. We can fix this using a regular expression.

In [38]:
skater_stats['name'] = skater_stats['name'].str.replace(r'^\d+\s*([^\d]+)$', r'\1', regex=True)

Once again, we need to group our positions appropriately. On MoneyPuck, Forwards are divided into C, R, and L. We'll convert all of these to 'F'.

In [39]:
skater_stats['pos'] = skater_stats['pos'].replace(['C','R','L'], 'F')

Before finalizing this dataframe, let's rename a couple columns. While it is informative to have '(minutes)' displayed in the icetime column, we don't want the header to contain parentheses. We're also going to stick the the convention of player names being contained in a column called `player`.

In [40]:
skater_stats = skater_stats.rename(columns={'icetime_(minutes)': 'icetime', 'name': 'player'})

In [41]:
# Reset dataframe index
skater_stats.reset_index(drop=True, inplace=True)

In [42]:
# Check the final dataframe
skater_stats

,player,pos,season,team,games_played,icetime,expected_goals,goals,assists,points,...,rebounds_created_above_expected,xgoals_on_rebounds_shots,share_of_xgoals_from_rebounds_shots,xgoals_from_non_rebounds,xgoals_of_expected_rebounds,created_xgoals,created_xgoals_minus_actual_xgoals,shooting_talent_above_average,shooting_talent_adjusted_expected_goals,goals_above_shooting_talent
0,Frans Nielsen,F,2010-11,NYI,71,1261,14.7,13,31,44,...,1.4,2.4,0.162,12.4,2.1,14.4,-0.3,-0.170,12.2,0.8
1,Jaroslav Spacek,D,2010-11,MTL,59,1135,2.7,1,15,16,...,-2.5,0.1,0.022,2.6,0.9,3.5,0.8,0.000,2.7,-1.7
2,Antti Miettinen,F,2010-11,MIN,73,1242,15.7,16,19,35,...,-8.4,1.2,0.076,14.5,2.6,17.1,1.4,0.006,15.8,0.2
3,Kyle Quincey,D,2010-11,COL,21,411,1.4,0,1,1,...,2.9,0.1,0.036,1.3,0.5,1.8,0.4,-0.226,1.1,-1.1
4,Sergei Samsonov,F,2010-11,FLA,78,1207,15.6,13,27,40,...,-3.3,4.4,0.282,11.2,2.1,13.3,-2.3,0.044,16.2,-3.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11677,Rasmus Asplund,F,2022-23,NSH,46,504,5.9,2,6,8,...,1.6,2.4,0.414,3.4,0.7,4.2,-1.7,-0.216,4.6,-2.6
11678,Ross Johnston,F,2022-23,NYI,16,124,0.3,0,2,2,...,0.8,0.0,0.000,0.3,0.1,0.4,0.1,0.000,0.3,-0.3
11679,Neal Pionk,D,2022-23,WPG,82,1799,7.0,10,23,33,...,-1.4,0.7,0.097,6.3,1.8,8.1,1.1,-0.125,6.1,3.9
11680,Tyler Pitlick,F,2022-23,STL,61,614,5.6,7,9,16,...,6.3,1.7,0.303,3.9,0.8,4.7,-0.9,0.038,5.9,1.1


In [43]:
# Save cleaned skater_stats dataframe
skater_stats.to_csv('../data/skater_stats_cleaned.csv', index=False)

### Goalie Stats

Much of the process for getting goalie stats from MoneyPuck is similar to what we did above for skate stats, however, there are a few key differences. For one, since they are contained within their own webpage, MoneyPucl does not both to havea position column, so we need to create this manually and fill it with 'G'.

In [44]:
driver.quit()
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

In [45]:
driver.get("https://moneypuck.com/goalies.htm")
time.sleep(3)

# Initiate final dataframe to store all player stats
goalie_stats = pd.DataFrame()

# Select only Regular Season Stats
select_element('//*[@id="table_playoff_type"]/option[1]')

# Iterate through all seasons from 2010-11 to 2021-22
for season_str, season in seasons.items():
    # Select Season
    select_element(f'//*[@id="season_type"]/option[{season}]')

    # Save the stas from the current season
    time.sleep(8)
    table_element = driver.find_element(By.XPATH, '//*[@id="goaliesTable"]')
    table_data = pd.read_html(table_element.get_attribute('outerHTML'))
    season_stats = pd.DataFrame(table_data[0])
    
    # Get team names from logo attributes
    tr_elements = driver.find_elements(By.XPATH, '//*[@id="goaliesTable"]/tbody/tr')
    alt_values = [tr_element.find_element(By.XPATH, './th/table/tbody/tr/td[2]/img').get_attribute('alt') for tr_element in tr_elements]

    # Convert headers to snake-case
    season_stats.columns = season_stats.columns.str.lower().str.replace(' ', '_').str.replace('%','pct')
    
    # The produced dataframe added a null row every other row, we'll drop those using the games_played column
    season_stats.dropna(subset=['games_played'], inplace=True)

    # Add position, season, and team columns
    season_stats['pos'] = 'G'
    season_stats['season'] = season_str
    season_stats['team'] = alt_values
    season_stats['team'] = season_stats['team'].map(teams_dict)
    
    # Move new columns to the beginning of the dataframe for clarity
    pos = season_stats.pop('pos')
    season = season_stats.pop('season')
    team = season_stats.pop('team')
    season_stats.insert(1, 'pos', pos)
    season_stats.insert(2, 'season', season)
    season_stats.insert(3, 'team', team)
    
    # Print shape of each season to view scraping progress
    print(f'Shape of {season_str} stats: {season_stats.shape}')
    
    # Append each season to the main stats dataframe
    goalie_stats = pd.concat([goalie_stats, season_stats])

Shape of 2010-11 stats: (87, 41)
Shape of 2011-12 stats: (88, 41)
Shape of 2012-13 stats: (82, 41)
Shape of 2013-14 stats: (97, 41)
Shape of 2014-15 stats: (92, 41)
Shape of 2015-16 stats: (92, 41)
Shape of 2016-17 stats: (94, 41)
Shape of 2017-18 stats: (95, 41)
Shape of 2018-19 stats: (93, 41)
Shape of 2019-20 stats: (85, 41)
Shape of 2020-21 stats: (98, 41)
Shape of 2021-22 stats: (119, 41)
Shape of 2022-23 stats: (107, 41)


In [46]:
goalie_stats

,name,pos,season,team,games_played,goals_against,expected_goals_against,goals_saved_above_expected,goals_saved_above_expected_per_60,save_pct_on_unblocked_shots,...,on_goal_pct_above_expected,low_danger_unblocked_shot_attempt_save_pct,xlow_danger_unblocked_shot_attempt_save_pct,low_danger_unblocked_shot_attempt_savepct_above_expected,medium_danger_unblocked_shot_attempt_save_pct,xmedium_danger_unblocked_shot_attempt_save_pct,medium_danger_unblocked_shot_attemptsave_pct_above_expected,high_danger_unblocked_shot_attempt_save_pct,xhigh_danger_unblocked_shot_attempt_save_pct,high_danger_unblocked_shot_attempt_save_pct_above_expected
0,1Tim Thomas,G,2010-11,BOS,57.0,112,151.56,39.6,0.706,0.966,...,1.77%,0.982,0.971,0.011,0.914,0.882,0.032,0.691,0.651,0.040
2,2Cam Ward,G,2010-11,CAR,74.0,184,209.88,25.9,0.365,0.959,...,-1.88%,0.978,0.971,0.007,0.894,0.880,0.015,0.659,0.672,-0.013
4,3Jonas Hiller,G,2010-11,ANA,49.0,114,134.72,20.7,0.465,0.959,...,-0.58%,0.980,0.970,0.010,0.895,0.879,0.016,0.648,0.664,-0.015
6,4Carey Price,G,2010-11,MTL,72.0,165,185.54,20.5,0.292,0.959,...,1.09%,0.978,0.972,0.006,0.907,0.880,0.027,0.626,0.674,-0.048
8,5Roberto Luongo,G,2010-11,VAN,60.0,126,144.60,18.6,0.312,0.960,...,1.88%,0.978,0.971,0.007,0.916,0.879,0.036,0.593,0.678,-0.085
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
204,103Jonathan Quick,G,2022-23,VGK,41.0,127,109.83,-17.2,-0.462,0.939,...,-1.25%,0.958,0.971,-0.012,0.846,0.876,-0.031,0.730,0.671,0.059
206,104Jack Campbell,G,2022-23,EDM,36.0,115,96.72,-18.3,-0.542,0.935,...,1.73%,0.960,0.969,-0.010,0.866,0.879,-0.013,0.604,0.655,-0.051
208,105Spencer Martin,G,2022-23,VAN,29.0,107,83.47,-23.5,-0.876,0.929,...,-0.29%,0.941,0.968,-0.027,0.884,0.876,0.008,0.651,0.690,-0.038
210,106Kaapo Kahkonen,G,2022-23,SJS,37.0,135,110.42,-24.6,-0.701,0.938,...,-0.03%,0.962,0.970,-0.008,0.825,0.877,-0.052,0.706,0.690,0.017


On the goalies web page, the Atlanta Thrashers and Seattle Kraken are listed as `undefined` in their respective logo atrributes. When we try to pull the team names, they get nullified. We need to fill these `NaN` values with the appropriate team names. Fortunately, these two teams did not exist at the same time in the NHL, so the best way to do this is to map the team name based on the season the stats were pulled from. For players with a `NaN` team name who played in the 2010-11 season, we know that player must have played for the Atlanta Thrashers. Likewise, a player with a `NaN` team name who played in 2021-22 season must have played for the Seattle Kraken. We can now fill these nulls appropriately.

In [47]:
# Fill null team names with appropriate team name
goalie_stats['team'].fillna(
    goalie_stats['season'].map({'2010-11': 'ATL', '2021-22': 'SEA'}),
    inplace=True
)

In [48]:
# Convert percentage strings to numeric values
pct_cols = [col for col in goalie_stats.columns if 'pct' in col]
for col in pct_cols:
    if goalie_stats[col].dtype == 'object':
        goalie_stats[col] = goalie_stats[col].str.strip('%').astype(float) / 100

In [49]:
# Look to see what other object columns are in the dataframe
goalie_stats.select_dtypes(include=['object']).columns.tolist()

['name', 'pos', 'season', 'team', 'goals_against']

We know that `goals_against` should be an integer values, so we'll go ahead an concert it.

In [50]:
goalie_stats['goals_against'] = goalie_stats['goals_against'].astype(int)

Once again, we need to fix the player names.

In [51]:
goalie_stats['name'] = goalie_stats['name'].str.replace(r'^\d+\s*([^\d]+)$', r'\1', regex=True)

Below, we'll convert the round float values to integers once more.

In [52]:
int_cols = ['games_played','icetime_(minutes)','puck_freezes',
            'saves_on_shots_on_goal','saves_on_unblocked_shot_attempts']

for col in int_cols:
    goalie_stats[col] = goalie_stats[col].astype(int)

We have one duplicated column in this dataframe too. Let's remove it.

In [53]:
goalie_stats = goalie_stats.drop(columns='goals_against.1')

Let's keep things consistent and rename our columns appropriately.

In [54]:
goalie_stats = goalie_stats.rename(columns={'icetime_(minutes)': 'icetime', 'name': 'player'})

In [55]:
# Reset dataframe index
goalie_stats.reset_index(drop=True, inplace=True)

In [56]:
# Check final dataframe
goalie_stats

,player,pos,season,team,games_played,goals_against,expected_goals_against,goals_saved_above_expected,goals_saved_above_expected_per_60,save_pct_on_unblocked_shots,...,on_goal_pct_above_expected,low_danger_unblocked_shot_attempt_save_pct,xlow_danger_unblocked_shot_attempt_save_pct,low_danger_unblocked_shot_attempt_savepct_above_expected,medium_danger_unblocked_shot_attempt_save_pct,xmedium_danger_unblocked_shot_attempt_save_pct,medium_danger_unblocked_shot_attemptsave_pct_above_expected,high_danger_unblocked_shot_attempt_save_pct,xhigh_danger_unblocked_shot_attempt_save_pct,high_danger_unblocked_shot_attempt_save_pct_above_expected
0,Tim Thomas,G,2010-11,BOS,57,112,151.56,39.6,0.706,0.966,...,0.0177,0.982,0.971,0.011,0.914,0.882,0.032,0.691,0.651,0.040
1,Cam Ward,G,2010-11,CAR,74,184,209.88,25.9,0.365,0.959,...,-0.0188,0.978,0.971,0.007,0.894,0.880,0.015,0.659,0.672,-0.013
2,Jonas Hiller,G,2010-11,ANA,49,114,134.72,20.7,0.465,0.959,...,-0.0058,0.980,0.970,0.010,0.895,0.879,0.016,0.648,0.664,-0.015
3,Carey Price,G,2010-11,MTL,72,165,185.54,20.5,0.292,0.959,...,0.0109,0.978,0.972,0.006,0.907,0.880,0.027,0.626,0.674,-0.048
4,Roberto Luongo,G,2010-11,VAN,60,126,144.60,18.6,0.312,0.960,...,0.0188,0.978,0.971,0.007,0.916,0.879,0.036,0.593,0.678,-0.085
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1224,Jonathan Quick,G,2022-23,VGK,41,127,109.83,-17.2,-0.462,0.939,...,-0.0125,0.958,0.971,-0.012,0.846,0.876,-0.031,0.730,0.671,0.059
1225,Jack Campbell,G,2022-23,EDM,36,115,96.72,-18.3,-0.542,0.935,...,0.0173,0.960,0.969,-0.010,0.866,0.879,-0.013,0.604,0.655,-0.051
1226,Spencer Martin,G,2022-23,VAN,29,107,83.47,-23.5,-0.876,0.929,...,-0.0029,0.941,0.968,-0.027,0.884,0.876,0.008,0.651,0.690,-0.038
1227,Kaapo Kahkonen,G,2022-23,SJS,37,135,110.42,-24.6,-0.701,0.938,...,-0.0003,0.962,0.970,-0.008,0.825,0.877,-0.052,0.706,0.690,0.017


In [57]:
# Save cleaned goalie_stats dataframe
goalie_stats.to_csv('../data/goalie_stats_cleaned.csv', index=False)

---
## Scraping & Cleaning: Team Standings

The last data we need to scrape if we want to get a sense of how a player's salary can be impcated by the performance of their team is standings data. Rather than trying to arbitrarily define team perfomance by team stats like Goals For or Goals Against, we're going to simply consider the final standings for each season. This way, we can easily link a player to their team's performance through the team name.

Scraping this data through Hockey Reference is fairly straightforward. We start by having the web driver get the web page for the first season (2010-11). Rather than looping through a list of possible elements to select, we can get the data we need from the appropriate table, then simply click on "Next Season >>" to load up the next page. One fail safe that needed to be built into the code was tool to get past a pop-up that occasionally appears when using the Hockey Reference website. After some time on the website, this pop-up may appear to gauge the user's experience with the webpage, however, it can block the "Next Season >>" button, rednering the webdriver incapable of progressing through the seasons.

<img src="../assets/hockey-ref-next-season.png" alt="hockey-ref-next-seasonn" style="width:350px;height:175px;">

Considering the simplicity of the dataset, all of the cleaning steps for this dataframe are built into the scraping code. The headers are lowercased, and columns are renamed and reordered.

In [58]:
driver.get('https://www.hockey-reference.com/leagues/NHL_2011_standings.html')
time.sleep(2)

# Initiate final dataframe to store team standings for each year
team_standings = pd.DataFrame()


for season_str, season in seasons.items():
    
    # Save results for the current season
    table_element = driver.find_element(By.XPATH, '//*[@id="expanded_standings"]')
    table_data = pd.read_html(table_element.get_attribute('outerHTML'))
    season_standings = pd.DataFrame(table_data[0])
    season_standings['season'] = season_str
    # Click to next season, otherwise exite out of pop-up and continue
    try:
        select_element('//*[@id="meta"]/div[2]/div/a[2]')
    except ElementClickInterceptedException:
        select_element('//*[@id="modal-close"]')
    time.sleep(5)
    
    # Lowercase headers, rename and reorder columns
    season_standings.columns = season_standings.columns.str.lower()
    season_standings = season_standings.loc[:, ['rk', 'unnamed: 1', 'season']]
    season_standings = season_standings.rename(columns={'rk': 'final_standing','unnamed: 1': 'team'})
    season_standings['team'] = season_standings['team'].map(teams_dict)
    season_standings = season_standings[['team','season','final_standing']]
    
    # Append each season to the main standings dataframe
    team_standings = pd.concat([team_standings, season_standings])

In [59]:
# Reset the dataframe index
team_standings.reset_index(drop=True, inplace=True)

In [60]:
# Check the final dataframe
team_standings

,team,season,final_standing
0,VAN,2010-11,1
1,WSH,2010-11,2
2,PIT,2010-11,3
3,PHI,2010-11,4
4,SJS,2010-11,5
...,...,...,...
391,NJD,2022-23,28
392,PHI,2022-23,29
393,SEA,2022-23,30
394,ARI,2022-23,31


In [61]:
# Save the cleaned team_standings dataframe
team_standings.to_csv('../data/team_standings_cleaned.csv', index=False)

---

## Merging Data

In this section, we're going to merge our cleaned dataframes into individual dataframes for each position, containing all of the necessary columns to perform our data analysis and modeling. The head of each cleaned dataframe is displayed as a reference.

In [62]:
signings.head(2)

,player,pos,contract_aav,season
0,Dana Tyrell,F,600000,2010-11
1,Kyle Turris,F,1400000,2010-11


In [63]:
salary_cap.head(2)

,season,pct_change,upper_limit,lower_limit,min_salary
2,2022-23,0.0121,83500000,61700000,775000
3,2021-22,0.0123,82500000,61000000,750000


In [64]:
skater_stats.head(2)

,player,pos,season,team,games_played,icetime,expected_goals,goals,assists,points,...,rebounds_created_above_expected,xgoals_on_rebounds_shots,share_of_xgoals_from_rebounds_shots,xgoals_from_non_rebounds,xgoals_of_expected_rebounds,created_xgoals,created_xgoals_minus_actual_xgoals,shooting_talent_above_average,shooting_talent_adjusted_expected_goals,goals_above_shooting_talent
0,Frans Nielsen,F,2010-11,NYI,71,1261,14.7,13,31,44,...,1.4,2.4,0.162,12.4,2.1,14.4,-0.3,-0.17,12.2,0.8
1,Jaroslav Spacek,D,2010-11,MTL,59,1135,2.7,1,15,16,...,-2.5,0.1,0.022,2.6,0.9,3.5,0.8,0.00,2.7,-1.7


In [65]:
goalie_stats.head(2)

,player,pos,season,team,games_played,goals_against,expected_goals_against,goals_saved_above_expected,goals_saved_above_expected_per_60,save_pct_on_unblocked_shots,...,on_goal_pct_above_expected,low_danger_unblocked_shot_attempt_save_pct,xlow_danger_unblocked_shot_attempt_save_pct,low_danger_unblocked_shot_attempt_savepct_above_expected,medium_danger_unblocked_shot_attempt_save_pct,xmedium_danger_unblocked_shot_attempt_save_pct,medium_danger_unblocked_shot_attemptsave_pct_above_expected,high_danger_unblocked_shot_attempt_save_pct,xhigh_danger_unblocked_shot_attempt_save_pct,high_danger_unblocked_shot_attempt_save_pct_above_expected
0,Tim Thomas,G,2010-11,BOS,57,112,151.56,39.6,0.706,0.966,...,0.0177,0.982,0.971,0.011,0.914,0.882,0.032,0.691,0.651,0.040
1,Cam Ward,G,2010-11,CAR,74,184,209.88,25.9,0.365,0.959,...,-0.0188,0.978,0.971,0.007,0.894,0.880,0.015,0.659,0.672,-0.013


In [66]:
team_standings.head(2)

,team,season,final_standing
0,VAN,2010-11,1
1,WSH,2010-11,2


We'll start by creating a `skaters` dataframe that contains information for both Forwards and Defense, and we'll split it later. First, we can merge `skater_signings` with the salary cap based on the season. We're doing a left merge because we want to retain unique player columns and simply add the salary cap details for each season to every signing. After that, we can merge the `skater_stats`. We want to merge on `player`, `pos`, and `season` so we can retain unique rows for players who signed contracts in multiple years. To finish it off, we'll tack on the `team_Standings` data based on the team and the season so we know how a player's team finished specifically the season before they signed their contract.

In [67]:
# Merge salary_cap info with signings
skaters = signings_skaters.merge(salary_cap, on='season', how='left')

# Merge skater stats
skaters = skaters.merge(skater_stats, on=['player', 'pos', 'season'], how='left')

# Merge team standings
skaters = skaters.merge(team_standings, on=['team', 'season'], how='left')

Now that we have a combined dataframe, let's check for any null values.

In [68]:
skaters.isna().sum().sort_values(ascending = False).loc[lambda x: x > 0]

final_standing                         131
shooting_pct                           125
on-ice_goals_pct                       124
share_of_xgoals_from_rebounds_shots    123
net_miss_pct_above_expected            123
                                      ... 
assists_per_60_minutes                 119
giveaways                              119
shots_on_goal_per_60_minutes           119
shot_attempts_per_60_minutes           119
games_played                           119
Length: 76, dtype: int64

In [69]:
skaters.shape

(1094, 84)

While the preference would be to impute missing values where possible, it's important to note that the number of null values is nearly the same across almost all columns. Essentially, there are just over 130 rows that are completely empty. With over 70 different features, selectively imputing the mean or median across all columns for all empty rows could significantly impact the results given the size of our dataset. Instead, we will choose to drop these null rows and accept a potential decrease in modeling performance.

In [70]:
skaters.dropna(inplace=True)

In [71]:
skaters.head()

,player,pos,contract_aav,season,pct_change,upper_limit,lower_limit,min_salary,team,games_played,...,xgoals_on_rebounds_shots,share_of_xgoals_from_rebounds_shots,xgoals_from_non_rebounds,xgoals_of_expected_rebounds,created_xgoals,created_xgoals_minus_actual_xgoals,shooting_talent_above_average,shooting_talent_adjusted_expected_goals,goals_above_shooting_talent,final_standing
0,Dana Tyrell,F,600000,2010-11,0.0825,64300000,48300000,525000,TBL,78.0,...,0.9,0.129,6.2,1.0,7.2,0.1,0.000,7.1,-1.1,8.0
1,Kyle Turris,F,1400000,2010-11,0.0825,64300000,48300000,525000,ARI,65.0,...,1.7,0.163,8.5,1.6,10.0,-0.1,0.000,10.1,0.9,11.0
2,Shawn Thornton,F,1100000,2010-11,0.0825,64300000,48300000,525000,BOS,79.0,...,0.5,0.055,9.2,1.7,10.9,1.2,-0.250,7.3,2.7,7.0
3,Daniel Carcillo,F,825000,2010-11,0.0825,64300000,48300000,525000,PHI,57.0,...,0.9,0.216,3.2,0.7,3.9,-0.2,-0.076,3.8,0.2,4.0
4,Mikhail Grabovski,F,5500000,2010-11,0.0825,64300000,48300000,525000,TOR,81.0,...,2.8,0.114,21.8,3.9,25.8,1.1,-0.075,22.8,6.2,22.0


In [72]:
# Save cleaned and merged skaters dataframe
skaters.to_csv('../data/skaters_cleaned.csv', index=False)

With the skaters dataframe completelty megred and cleaned, we can go ahead and split it into two separate dataframes for Forwards and Defense.

In [73]:
# Create dataframe containing only Forwards
forwards = skaters[skaters['pos'] == 'F']

In [74]:
forwards.shape

(464, 84)

In [75]:
forwards.head()

,player,pos,contract_aav,season,pct_change,upper_limit,lower_limit,min_salary,team,games_played,...,xgoals_on_rebounds_shots,share_of_xgoals_from_rebounds_shots,xgoals_from_non_rebounds,xgoals_of_expected_rebounds,created_xgoals,created_xgoals_minus_actual_xgoals,shooting_talent_above_average,shooting_talent_adjusted_expected_goals,goals_above_shooting_talent,final_standing
0,Dana Tyrell,F,600000,2010-11,0.0825,64300000,48300000,525000,TBL,78.0,...,0.9,0.129,6.2,1.0,7.2,0.1,0.000,7.1,-1.1,8.0
1,Kyle Turris,F,1400000,2010-11,0.0825,64300000,48300000,525000,ARI,65.0,...,1.7,0.163,8.5,1.6,10.0,-0.1,0.000,10.1,0.9,11.0
2,Shawn Thornton,F,1100000,2010-11,0.0825,64300000,48300000,525000,BOS,79.0,...,0.5,0.055,9.2,1.7,10.9,1.2,-0.250,7.3,2.7,7.0
3,Daniel Carcillo,F,825000,2010-11,0.0825,64300000,48300000,525000,PHI,57.0,...,0.9,0.216,3.2,0.7,3.9,-0.2,-0.076,3.8,0.2,4.0
4,Mikhail Grabovski,F,5500000,2010-11,0.0825,64300000,48300000,525000,TOR,81.0,...,2.8,0.114,21.8,3.9,25.8,1.1,-0.075,22.8,6.2,22.0


In [76]:
# Save cleaned and merged forwards dataframe
forwards.to_csv('../data/forwards_cleaned.csv', index=False)

In [77]:
# Create dataframe containing only Defense
defense = skaters[skaters['pos'] == 'D']

In [78]:
defense.shape

(488, 84)

In [79]:
defense.head()

,player,pos,contract_aav,season,pct_change,upper_limit,lower_limit,min_salary,team,games_played,...,xgoals_on_rebounds_shots,share_of_xgoals_from_rebounds_shots,xgoals_from_non_rebounds,xgoals_of_expected_rebounds,created_xgoals,created_xgoals_minus_actual_xgoals,shooting_talent_above_average,shooting_talent_adjusted_expected_goals,goals_above_shooting_talent,final_standing
532,Drew Doughty,D,7000000,2010-11,0.0825,64300000,48300000,525000,LAK,76.0,...,0.1,0.005,9.4,2.3,11.6,2.2,0.136,10.7,0.3,12.0
533,Chris Campoli,D,1750000,2010-11,0.0825,64300000,48300000,525000,CHI,77.0,...,0.4,0.097,3.5,1.0,4.4,0.6,0.118,4.3,-0.3,13.0
534,Tyler Myers,D,5500000,2010-11,0.0825,64300000,48300000,525000,BUF,80.0,...,0.7,0.095,7.1,1.8,8.9,1.1,0.162,9.1,0.9,15.0
535,Luke Schenn,D,3600000,2010-11,0.0825,64300000,48300000,525000,TOR,82.0,...,0.3,0.063,4.3,1.5,5.8,1.2,-0.045,4.4,0.6,22.0
537,Braydon Coburn,D,4500000,2010-11,0.0825,64300000,48300000,525000,PHI,82.0,...,0.5,0.109,3.8,1.4,5.2,0.9,0.000,4.3,-2.3,4.0


In [80]:
# Save cleaned and merged defense dataframe
defense.to_csv('../data/defense_cleaned.csv', index=False)

We can repeat the same steps from above with the goalie data.

In [81]:
# Merge salary_cap info with signings
goalies = signings_goalies.merge(salary_cap, on='season', how='left')

# Merge goalie stats
goalies = goalies.merge(goalie_stats, on=['player', 'pos', 'season'], how='left')

# Merge team standings
goalies = goalies.merge(team_standings, on=['team', 'season'], how='left')

In [82]:
goalies.isna().sum().sort_values(ascending = False).loc[lambda x: x > 0]

high_danger_unblocked_shot_attempt_save_pct_above_expected     42
xhigh_danger_unblocked_shot_attempt_save_pct                   42
high_danger_unblocked_shot_attempt_save_pct                    42
medium_danger_unblocked_shot_attemptsave_pct_above_expected    42
xmedium_danger_unblocked_shot_attempt_save_pct                 42
medium_danger_unblocked_shot_attempt_save_pct                  42
rebounds_per_save                                              41
pct_of_shot_attempts_blocked_by_teammates                      41
puck_freezes                                                   41
expected_puck_freeze                                           41
puck_freezes_above_expected                                    41
puck_freezes_above_expected_per_shot_on_goal                   41
saves_on_shots_on_goal                                         41
saves_on_unblocked_shot_attempts                               41
on_goal_pct_above_expected                                     41
pct_of_unb

In [83]:
goalies.shape

(230, 46)

In [84]:
goalies.dropna(inplace=True)

In [85]:
goalies.shape

(187, 46)

In [86]:
goalies.head()

,player,pos,contract_aav,season,pct_change,upper_limit,lower_limit,min_salary,team,games_played,...,low_danger_unblocked_shot_attempt_save_pct,xlow_danger_unblocked_shot_attempt_save_pct,low_danger_unblocked_shot_attempt_savepct_above_expected,medium_danger_unblocked_shot_attempt_save_pct,xmedium_danger_unblocked_shot_attempt_save_pct,medium_danger_unblocked_shot_attemptsave_pct_above_expected,high_danger_unblocked_shot_attempt_save_pct,xhigh_danger_unblocked_shot_attempt_save_pct,high_danger_unblocked_shot_attempt_save_pct_above_expected,final_standing
0,Ilya Bryzgalov,G,5666666,2010-11,0.0825,64300000,48300000,525000,ARI,68.0,...,0.976,0.972,0.005,0.895,0.878,0.018,0.624,0.667,-0.043,11.0
1,Henrik Karlsson,G,862500,2010-11,0.0825,64300000,48300000,525000,CGY,17.0,...,0.965,0.971,-0.007,0.901,0.875,0.026,0.567,0.639,-0.072,17.0
2,Pekka Rinne,G,7000000,2010-11,0.0825,64300000,48300000,525000,NSH,64.0,...,0.977,0.972,0.006,0.887,0.880,0.007,0.683,0.674,0.008,10.0
3,Brian Elliott,G,1800000,2010-11,0.0825,64300000,48300000,525000,COL,55.0,...,0.969,0.972,-0.002,0.839,0.877,-0.038,0.614,0.647,-0.033,29.0
4,Ben Bishop,G,650000,2010-11,0.0825,64300000,48300000,525000,STL,7.0,...,0.972,0.971,0.000,0.784,0.874,-0.090,0.667,0.642,0.024,20.0


In [87]:
# Save cleaned and merged goalies dataframe
goalies.to_csv('../data/goalies_cleaned.csv', index=False)

---
## Active Player Contracts & 2022-23 Stats

We've collected all of the data we need for the modeling process, but we need a little more data. In order to develop a web app that compares a player's predicted contract value to their current salary, we need to get a dataset of active NHL contracts. To do this, we're going to go back to CapFriendly. While we have scraped from CapFriendly already, the webpage for active contracts has a very differnt layout than what we've seen so far. Data can be filtered and then a user must click through pages using a tool at the bottom of the web page.

<img src="../assets/contract-pages.png" alt="contract-pages" style="width:350px;height:100px;">

Rather than use the webdriver to scroll to the bottom of the page and click through to the next page, the easiest way to scrape this data is to select the filtering parameters that we want to use, update the results, and then loop through different pages via the url extension. Before we begin, we'll take a moment to reboot the driver again.

In [88]:
driver.quit()
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

As mentioned above, we are looping through the url extensions for each page by appending the extension to the end of the url string. The dictionary below pairs each page with its respective url extension.

In [89]:
pages = {
    '1': '',
    '2': '&pg=2',
    '3': '&pg=3',
    '4': '&pg=4',
    '5': '&pg=5',
    '6': '&pg=6',
    '7': '&pg=7',
    '8': '&pg=8',
    '9': '&pg=9',
    '10': '&pg=10',
    '11': '&pg=11',
    '12': '&pg=12',
    '13': '&pg=13',
    '14': '&pg=14',
    '15': '&pg=15',
    '16': '&pg=16',
    '17': '&pg=17',
    '18': '&pg=18',
    '19': '&pg=19',
    '20': '&pg=20',
    '21': '&pg=21',
    '22': '&pg=22',
    '23': '&pg=23',
    '24': '&pg=24',
    '25': '&pg=25',
    '26': '&pg=26',
    '27': '&pg=27',
    '28': '&pg=28',
    '29': '&pg=29',
    '30': '&pg=30',
    '31': '&pg=31',
    '32': '&pg=32'
}

In [90]:
# Define the base url withr the necessary filtering parameters have been selected
base_url = 'https://www.capfriendly.com/browse/active?stats-season=2023&hide=clauses,age,handed,expiry-status,caphit,skater-stats,goalie-stats'

# Create an empty dataframe to store all of the contracts
active_contracts = pd.DataFrame(columns=['player', 'pos', 'team', 'salary'])

# Loop through the url extensions defined above
for page, element in pages.items():

    # Load the website on the current page
    driver.get(base_url + element)
    time.sleep(2)
    
    # Scroll the page just enough that the driver can identify the table
    scroll_script = f"window.scrollBy(0, 800);"
    driver.execute_script(scroll_script)
    
    # Save the data from the current page
    time.sleep(2)
    table_element = driver.find_element(By.XPATH, '//*[@id="brwt"]')
    time.sleep(2)
    table_data = pd.read_html(table_element.get_attribute('outerHTML'))
    page_data = pd.DataFrame(table_data[0])
    page_data.columns = page_data.columns.str.lower()
    
    # Append the page data to the main contracts dataframe
    active_contracts = pd.concat([active_contracts, page_data], join='inner')


In [91]:
active_contracts

,player,pos,team,salary
0,1. Connor McDavid,C,EDM,"$12,000,000"
1,2. Artemi Panarin,LW,NYR,"$12,500,000"
2,3. Auston Matthews,C,TOR,"$7,950,000"
3,4. Erik Karlsson,RD,SJS,"$12,000,000"
4,5. Drew Doughty,RD,LAK,"$11,000,000"
...,...,...,...,...
37,1588. Keaton Middleton,LD,COL,"$750,000"
38,1589. Olli Juolevi,LD,ANA,"$750,000"
39,1590. Jérémy Groleau,LD,NJD,"$750,000"
40,1591. Connor Ingram,G,ARI,"$750,000"


We have a few things we will do to this data before splitting it into separate dataframes by position. First, we need to clean up the names again. We'll use a regular expression for this. Next, we'll need to organize the positions, which are once again divided into C, LW, and RW for Forwards and LD, RD for Defense. Since we are only using the slaary for display purposes on the web app, we can leave it as a string.

In [92]:
# Fix player names by removing the numbers and period at the front
active_contracts['player'] = active_contracts['player'].str.replace(r'^\d+.\s*([^\d]+)$', r'\1', regex=True)

# Group positions by F, D, and G
active_contracts['pos'] = np.where(active_contracts['pos'].str.contains('D'), 'D',
                           np.where(active_contracts['pos'].str.contains('G'), 'G', 'F'))

In [93]:
# Define new dataframes containing active contracts by positon
active_contracts_f = active_contracts[active_contracts['pos'] == 'F']

active_contracts_d = active_contracts[active_contracts['pos'] == 'D']

active_contracts_g = active_contracts[active_contracts['pos'] == 'G']

Since this dataset will be searched by users of the web app, we need to make sure users can accurately identify the player they want to test with the app. To do so, we're going to combine the player's name, position, and team so a user can be sure they are selecting the correct player.

In [94]:
# Combine player names, positions, and team into a single column for each dataframe
active_contracts_f.loc[:, "player"] = active_contracts_f["player"] + ", " + active_contracts_f["pos"] + ", " + active_contracts_f["team"]
active_contracts_f = active_contracts_f.drop(columns=["pos", "team"])

active_contracts_d.loc[:, "player"] = active_contracts_d["player"] + ", " + active_contracts_d["pos"] + ", " + active_contracts_d["team"]
active_contracts_d = active_contracts_d.drop(columns=["pos", "team"])

active_contracts_g.loc[:, "player"] = active_contracts_g["player"] + ", " + active_contracts_g["pos"] + ", " + active_contracts_g["team"]
active_contracts_g = active_contracts_g.drop(columns=["pos", "team"])

In [95]:
# Check the new dataframe
active_contracts_f.head()

,player,salary
0,"Connor McDavid, F, EDM","$12,000,000"
1,"Artemi Panarin, F, NYR","$12,500,000"
2,"Auston Matthews, F, TOR","$7,950,000"
5,"John Tavares, F, TOR","$7,950,000"
6,"Mitchell Marner, F, TOR","$8,000,000"


### Player Stats 2022-23

If we want to test active players with the final models to predict their salaries, we need to separate out their stats from the latest season (2023). To do that we can merge player stats with the salary cap and team standings information. Since player signings are the taret variable for the model, we need to leave contract information out of these dataframes. The result should be a condensed dataframe containing all of the same variables that we will use to *train* the models, but only from the 2022-23 season.

In [96]:
# Merge skater stats and salary cap info
skaters_2023 = skater_stats.merge(salary_cap, on='season', how='left')

# Merge team standings
skaters_2023 = skaters_2023.merge(team_standings, on=['team', 'season'], how='left')

# Filter for 2022-23 season
skaters_2023 = skaters_2023[skaters_2023['season'] == '2022-23']

Again, we can split the skater data into forwards and defense.

In [97]:
# Define forwards dataframe for 2022-23 season
forwards_2023 = skaters_2023[skaters_2023['pos'] == 'F']

# Define defense dataframe for 2022-23 season
defense_2023 = skaters_2023[skaters_2023['pos'] == 'D']

We'll do the same for goalies.

In [98]:
# Merge goalie stats and salary cap info
goalies_2023 = goalie_stats.merge(salary_cap, on='season', how='left')

# Merge team standings
goalies_2023 = goalies_2023.merge(team_standings, on= ['team', 'season'], how='left')

# Filter for 2022-23 season
goalies_2023 = goalies_2023[goalies_2023['season'] == '2022-23']

In order to be able to link a user's input on the web app to a specific player to pass that player's stats through the model, we'll put it in the same format that we did for the `active_contracts` dataframe.

In [99]:
# Combine player names, positions, and team into a single column for each dataframe
forwards_2023.loc[:, "player"] = forwards_2023["player"] + ", " + forwards_2023["pos"] + ", " + forwards_2023["team"]
forwards_2023 = forwards_2023.drop(columns=["pos", "team", "season"])

defense_2023.loc[:, "player"] = defense_2023["player"] + ", " + defense_2023["pos"] + ", " + defense_2023["team"]
defense_2023 = defense_2023.drop(columns=["pos", "team", "season"])

goalies_2023.loc[:, "player"] = goalies_2023["player"] + ", " + goalies_2023["pos"] + ", " + goalies_2023["team"]
goalies_2023 = goalies_2023.drop(columns=["pos", "team", "season"])

In [100]:
# Reset index of each dataframe
forwards_2023.reset_index(drop=True, inplace=True)
defense_2023.reset_index(drop=True, inplace=True)
goalies_2023.reset_index(drop=True, inplace=True)

One last thing we need to do is make sure that the `active_contracts` dataframe and the stats dataframes contain the same list of players. To do this, we will drop players from the `active_contracts` dataframe that are not also in the stats dataframe, and vice versa.

In [101]:
active_contracts_f = active_contracts_f[active_contracts_f['player'].isin(forwards_2023['player'])]
active_contracts_f.shape

(534, 2)

In [102]:
active_contracts_d = active_contracts_d[active_contracts_d['player'].isin(defense_2023['player'])]
active_contracts_d.shape

(289, 2)

In [103]:
active_contracts_g = active_contracts_g[active_contracts_g['player'].isin(goalies_2023['player'])]
active_contracts_g.shape

(89, 2)

In [104]:
forwards_2023 = forwards_2023[forwards_2023['player'].isin(active_contracts_f['player'])]
forwards_2023.shape, active_contracts_f.shape

((532, 80), (534, 2))

One thing we notice her is that even after dropping players that don't appear in both, we have unbalanced dataframes. This tells us that we have some duplicate players. We'll make sure to drop duplicate players from each dataframe.

In [105]:
active_contracts_f = active_contracts_f.drop_duplicates()
forwards_2023.shape, active_contracts_f.shape

((532, 80), (532, 2))

In [106]:
defense_2023 = defense_2023[defense_2023['player'].isin(active_contracts_d['player'])]
defense_2023.shape, active_contracts_d.shape

((289, 80), (289, 2))

In [107]:
active_contracts_d = active_contracts_d.drop_duplicates()
defense_2023.shape, active_contracts_d.shape

((289, 80), (289, 2))

In [108]:
goalies_2023 = goalies_2023[goalies_2023['player'].isin(active_contracts_g['player'])]
goalies_2023.shape, active_contracts_g.shape

((88, 42), (89, 2))

In [109]:
active_contracts_g = active_contracts_g.drop_duplicates()
goalies_2023.shape, active_contracts_g.shape

((88, 42), (88, 2))

In [110]:
# Check the new dataframe
forwards_2023.head(3)

,player,games_played,icetime,expected_goals,goals,assists,points,primary_assists,secondary_assists,shifts,...,created_xgoals,created_xgoals_minus_actual_xgoals,shooting_talent_above_average,shooting_talent_adjusted_expected_goals,goals_above_shooting_talent,pct_change,upper_limit,lower_limit,min_salary,final_standing
0,"Ryan Reaves, F, MIN",73,680,6.7,5,10,15,5,5,889,...,5.6,-1.1,-0.125,5.9,-0.9,0.0121,83500000,61700000,775000,5.0
1,"Robby Fabbri, F, DET",28,447,6.8,7,9,16,5,4,504,...,5.6,-1.2,0.148,7.8,-0.8,0.0121,83500000,61700000,775000,25.0
2,"Michael Bunting, F, TOR",82,1295,28.0,23,26,49,19,7,1586,...,23.3,-4.7,0.043,29.2,-6.2,0.0121,83500000,61700000,775000,4.0


When passing the player stats through the model, it will be important that features are in the same order as when we train the model. The eastiest way to do this will be to order the columns alphabetically. We will do the same for the training data before training the model to make sure these different datasets are cohesive for the app.

In [111]:
ordered_columns = sorted(forwards_2023.columns)
forwards_2023 = forwards_2023.reindex(columns=ordered_columns)
defense_2023 = defense_2023.reindex(columns=ordered_columns)

ordered_columns = sorted(goalies_2023.columns)
goalies_2023 = goalies_2023.reindex(columns=ordered_columns)

In [112]:
forwards_2023.head(3)

,assists,assists_per_60_minutes,created_xgoals,created_xgoals_minus_actual_xgoals,defensive_zone_giveaways,expected_goals,expected_goals_per_60_minutes,expected_pct_of_unblocked_shots_that_missed_net,expected_shooting_pct_on_unblocked_shots,faceoff_win_pct,...,shots_on_goal,shots_on_goal_per_60_minutes,shots_that_missed_net,shots_that_were_blocked,takeaways,upper_limit,xgoals_from_non_rebounds,xgoals_of_expected_rebounds,xgoals_on_rebounds_shots,xrebounds_created
0,10,0.88,5.6,-1.1,10,6.7,0.59,0.290,0.087,0.267,...,48,4.23,21,10,15,83500000,4.8,0.8,1.9,3.8
1,9,1.21,5.6,-1.2,3,6.8,0.91,0.264,0.113,0.385,...,35,4.69,18,12,6,83500000,5.0,0.7,1.9,2.9
2,26,1.20,23.3,-4.7,14,28.0,1.30,0.249,0.111,0.500,...,174,8.06,79,45,49,83500000,19.9,3.5,8.2,15.4


In [113]:
# save active_contracts dataframes
active_contracts_f.to_csv('../data/active_contracts_f.csv', index=False)
active_contracts_d.to_csv('../data/active_contracts_d.csv', index=False)
active_contracts_g.to_csv('../data/active_contracts_g.csv', index=False)

# Save 2022-23 stats dataframes
forwards_2023.to_csv('../data/forwards_2023.csv', index=False)
defense_2023.to_csv('../data/defense_2023.csv', index=False)
goalies_2023.to_csv('../data/goalies_2023.csv', index=False)

---
#### Notebook Links

Part II - Exploratory Data Analysis (EDA)
- [`Part-2_eda.ipynb`](../code/Part-2_eda.ipynb)

Part III - Modeling
- [`Part-3.1_modeling-forwards.ipynb`](../code/Part-3.1_modeling-forwards.ipynb)
- [`Part-3.2_modeling-defense.ipynb`](../code/Part-3.2_modeling-defense.ipynb)
- [`Part-3.3_modeling-goalies.ipynb`](../code/Part-3.3_modeling-goalies.ipynb)

Part IV - Conclusion, Recommendations, and Sources
- [`Part-4_conclusion-and-recommendations.ipynb`](../code/Part-4_conclusion-and-recommendations.ipynb)